In [1]:
#Training a classifier
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import os

print(os.listdir("data/cell_images/"))

['Parasitized', 'Uninfected']


In [2]:
from torch.utils import data
import random

transform = transforms.Compose(
    [transforms.Resize((100,100)), 
     #transforms.ColorJitter(hue=0.05, saturation=0.05), 
     transforms.RandomRotation(30),
     transforms.RandomVerticalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#import data
dataset = datasets.ImageFolder(root = 'data/cell_images', transform=transform)

#loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2)

#image_means = torch.stack([t.mean(1).mean(1) for t, c in dataset])
#image_means.mean(0)

n = len(dataset)
n_val = int(n*0.1)  #nb of val elms
n_test = int(n*0.1) #nb of test elms
n_train = n-n_val-n_test

train_set, val_set, test_set = data.random_split(dataset, (n_train, n_val, n_test))

#define loaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=100, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, num_workers=0)

classes = ('Parasitized', 'Uninfected')

In [3]:
# Show some images, for testing

import matplotlib.pyplot as plt
import numpy as np
    
def imshow(img):
    img = img/2 + 0.5 #un-normalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
#get rd images
dataiter = iter(train_loader)
images, labels = dataiter.next()

#show imgs
imshow(torchvision.utils.make_grid(images))
#print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(20)))

<Figure size 640x480 with 1 Axes>

Parasitized Uninfected Parasitized Uninfected Parasitized Uninfected Parasitized Uninfected Uninfected Uninfected Uninfected Uninfected Parasitized Uninfected Uninfected Uninfected Parasitized Parasitized Parasitized Uninfected


In [29]:
#Define a CNN

import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #self.layer1 = nn.Sequential(
        self.conv1 = nn.Conv2d(3, 32, 3, stride = 1)
        #nn.ReLU(),
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        #)
        #self.layer2 = nn.Sequential(
        self.conv2 = nn.Conv2d(32, 32, kernel_size = 3, stride = 1)
        #nn.ReLU(),
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        #)
        #self.layer3 = nn.Sequential(
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1)
        #nn.ReLU(),
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        #)
        self.fc1 = nn.Linear(10*10*64, 64)
        self.fc2 = nn.Linear(64, 2)
        self.dropout = nn.Dropout()
    
    def forward(self, x):
        #print(x.shape)
        #out = self.pool1(F.relu(self.conv1(x)))
        #out = self.pool2(F.relu(self.conv2(out)))
        #out = self.pool3(F.relu(self.conv3(out)))
        out = F.relu(self.conv1(x))
        #print(out.shape)
        out = self.pool1(out)
        #print(out.shape)
        out = F.relu(self.conv2(out))
        #print(out.shape)
        out = self.pool2(out)
        #print(out.shape)
        out = F.relu(self.conv3(out))
        #print(out.shape)
        out = self.pool3(out)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.fc1(out)
        #print(out.shape)
        out = F.relu(out)
        #print(out.shape)
        out = self.dropout(out)
        #print(out.shape)
        out = self.fc2(out)
        #print(out.shape)
        #out = self.layer1(x)
        #out = self.layer2(out)
        #out = self.layer3(out)
        #out = out.view(out.size(0), -1)
        #out = self.layer4(out)
        return out

#cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ConvNet()
net.to(device)
print(net)

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6400, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.5)
)


In [36]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, weight_decay = 1e-4, momentum=0.9)
#lambda1 = lambda epoch: epoch // 30
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min',factor=0.1,patience=1)

In [ ]:
total_step = len(train_loader)
#loss_list = []
#acc_list = []
net = ConvNet()
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = 0.0
    val_loss = 0.0
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        #zero parameter grads
        optimizer.zero_grad()
        #scheduler.zero_grad()
        
        #Run fwd pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #av_loss += loss.item()*0.9+av_loss*0.1
        
        #Run bwd pass
        loss.backward()
        #scheduler.step()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i+1, running_loss/100))
            running_loss = 0

    scheduler.step(running_loss)      
print('Finished training')

[1,   100] loss: 0.693
[1,   200] loss: 0.693
[2,   100] loss: 0.692
[2,   200] loss: 0.693
[3,   100] loss: 0.693


In [156]:
print(torch.version.cuda)

None
